In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def evaluate_model(model, val_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())
    accuracy = accuracy_score(all_labels, all_predictions)
    return accuracy

def cross_validate(dataset, num_folds, num_epochs, batch_size, learning_rate):
    labels = [label for _, label in dataset]
    skf = StratifiedKFold(n_splits=num_folds)

    accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(dataset)), labels)):
        print(f"Fold {fold + 1}/{num_folds}")
        
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)
        
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        model = SimpleNN()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        train_model(model, train_loader, criterion, optimizer, num_epochs)
        
        accuracy = evaluate_model(model, val_loader)
        accuracies.append(accuracy)
        print(f"Validation Accuracy for Fold {fold + 1}: {accuracy:.4f}")

    return accuracies


In [2]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_size = int(0.8 * len(mnist_dataset))
val_size = len(mnist_dataset) - train_size
train_dataset, val_dataset = random_split(mnist_dataset, [train_size, val_size])

num_epochs = 5
batch_size = 64
learning_rate = 0.001
num_folds = 5

cross_validate(train_dataset, num_folds, num_epochs, batch_size, learning_rate)

final_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
final_model = SimpleNN()
final_optimizer = optim.Adam(final_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

train_model(final_model, final_train_loader, criterion, final_optimizer, num_epochs)

final_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
final_accuracy = evaluate_model(final_model, final_val_loader)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

Fold 1/5
Validation Accuracy for Fold 1: 0.9692
Fold 2/5
Validation Accuracy for Fold 2: 0.9632
Fold 3/5
Validation Accuracy for Fold 3: 0.9647
Fold 4/5
Validation Accuracy for Fold 4: 0.9720
Fold 5/5
Validation Accuracy for Fold 5: 0.9617
Final Validation Accuracy: 0.9714
